In [1]:
from src.optimisers.hadamard_gate_reduction import HadamardGateReduction
from src.optimisers import SingleQubitGateCancellation
from src.optimisers import CNOTGateCancellation
from src.converters.netlist_converters import qCircuit_to_netlist, netlist_to_qCircuit
from src.converters.graph_converter import create_graph_from_netlist, create_netlist_from_graph
from src.utils.graphs import plot_graph
from mpqp import QCircuit
from mpqp.gates import X, CNOT, Z, H, S, T
from mpqp.measures import BasisMeasure, ComputationalBasis, HadamardBasis
from mpqp.execution import *

In [4]:
def Sdg(qubit):
    sdg = S(qubit).inverse()
    sdg.label = "S†"
    return sdg

def Tdg(qubit):
    tdg = T(qubit).inverse()
    tdg.label = "T†"
    return tdg

In [5]:
def optimise_circuit(circ: QCircuit, print_graphs = False, do_h_red=True, do_sg_cancel=True, do_cnot_cancel=True):
    print("Circuit:")
    print(circ)
    netlist = qCircuit_to_netlist(circ)
    graph = create_graph_from_netlist(netlist)
    if print_graphs:
        print("First graph")
        plot_graph(graph, node_size=1000)
    if do_h_red:
        HadamardGateReduction.new_optimise(graph)
    if do_sg_cancel:
        SingleQubitGateCancellation.optimise(graph)
    if do_cnot_cancel:
        CNOTGateCancellation.optimise(graph)
    if print_graphs:
        print("Optimised graph")
        plot_graph(graph, node_size=1000)
    opt_circ = netlist_to_qCircuit(create_netlist_from_graph(graph))
    print("Optimised circuit")
    print(opt_circ)
    return opt_circ


In [6]:
def run_circuit(circ: QCircuit, nbqubits, shots=1000, basis=ComputationalBasis()):
    circ.add(BasisMeasure(list(range(nbqubits)), shots=shots, basis=basis))
    result = run(circ, IBMDevice.AER_SIMULATOR)
    print(result)

In [7]:
p_graphs = False

# Circuit 1

In [8]:
circ = QCircuit(1)
circ.add(H(0))
circ.add(S(0))
circ.add(H(0))
circ.add(S(0))
circ.add(H(0))

opt_circ = optimise_circuit(circ, p_graphs)

Circuit:
   ┌───┐┌───┐┌───┐┌───┐┌───┐
q: ┤ H ├┤ S ├┤ H ├┤ S ├┤ H ├
   └───┘└───┘└───┘└───┘└───┘
Optimised circuit
   ┌────┐
q: ┤ S† ├
   └────┘


In [7]:
# Should be 1
run_circuit(circ, 1)
run_circuit(opt_circ, 1)

Result: None, IBMDevice, AER_SIMULATOR
 Counts: [1000, 0]
 Probabilities: [1, 0]
 Samples:
  State: 0, Index: 0, Count: 1000, Probability: 1.0
 Error: None
Result: None, IBMDevice, AER_SIMULATOR
 Counts: [1000, 0]
 Probabilities: [1, 0]
 Samples:
  State: 0, Index: 0, Count: 1000, Probability: 1.0
 Error: None


# Circuit 2

In [9]:
circ = QCircuit(2)
circ.add(H(0))
circ.add(H(1))
circ.add(CNOT(0, 1))
circ.add(H(1))
circ.add(H(0))

optimise_circuit(circ, p_graphs)

Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘
Optimised circuit
     ┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──■──
          


QCircuit([CNOT(1,0)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 3

In [10]:
circ = QCircuit(2)
circ.add(H(1))
circ.add(S(1))
circ.add(CNOT(0, 1))
circ.add(Sdg(1))
circ.add(H(1))

optimise_circuit(circ, p_graphs)

Circuit:
                               
q_0: ────────────■─────────────
     ┌───┐┌───┐┌─┴─┐┌────┐┌───┐
q_1: ┤ H ├┤ S ├┤ X ├┤ S† ├┤ H ├
     └───┘└───┘└───┘└────┘└───┘
Optimised circuit
                     
q_0: ────────■───────
     ┌────┐┌─┴─┐┌───┐
q_1: ┤ S† ├┤ X ├┤ S ├
     └────┘└───┘└───┘


QCircuit([CustomGate(1), CNOT(0,1), S(1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 4

In [11]:
circ = QCircuit(2)
circ.add(Sdg(1))
circ.add(H(1))
circ.add(Sdg(1))
circ.add(CNOT(0, 1))
circ.add(S(1))
circ.add(H(1))
circ.add(S(1))

optimise_circuit(circ, p_graphs)

Circuit:
                                          
q_0: ───────────────────■─────────────────
     ┌────┐┌───┐┌────┐┌─┴─┐┌───┐┌───┐┌───┐
q_1: ┤ S† ├┤ H ├┤ S† ├┤ X ├┤ S ├┤ H ├┤ S ├
     └────┘└───┘└────┘└───┘└───┘└───┘└───┘
Optimised circuit
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘


QCircuit([CNOT(0,1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 5

In [12]:
circ = QCircuit(2)
circ.add(S(0))
circ.add(H(0))
circ.add(S(0))
circ.add(H(0))
circ.add(S(0))
circ.add(H(0))

circ.add(Sdg(1))
circ.add(H(1))
circ.add(Sdg(1))
circ.add(CNOT(0, 1))
circ.add(S(1))
circ.add(H(1))
circ.add(S(1))

optimise_circuit(circ, p_graphs)

Circuit:
     ┌───┐ ┌───┐┌───┐ ┌───┐┌───┐┌───┐                    
q_0: ┤ S ├─┤ H ├┤ S ├─┤ H ├┤ S ├┤ H ├──■─────────────────
     ├───┴┐├───┤├───┴┐└───┘└───┘└───┘┌─┴─┐┌───┐┌───┐┌───┐
q_1: ┤ S† ├┤ H ├┤ S† ├───────────────┤ X ├┤ S ├┤ H ├┤ S ├
     └────┘└───┘└────┘               └───┘└───┘└───┘└───┘
Optimised circuit
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘


QCircuit([CNOT(0,1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 6

In [13]:
circ = QCircuit(2)
circ.add(H(1))
circ.add(S(1))
circ.add(H(1))
circ.add(S(1))
circ.add(CNOT(0, 1))
circ.add(Sdg(1))
circ.add(H(1))

optimise_circuit(circ, p_graphs)

Circuit:
                                         
q_0: ──────────────────────■─────────────
     ┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌────┐┌───┐
q_1: ┤ H ├┤ S ├┤ H ├┤ S ├┤ X ├┤ S† ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘└────┘└───┘
Optimised circuit
                                
q_0: ─────────────■─────────────
     ┌────┐┌───┐┌─┴─┐┌────┐┌───┐
q_1: ┤ S† ├┤ H ├┤ X ├┤ S† ├┤ H ├
     └────┘└───┘└───┘└────┘└───┘


QCircuit([CustomGate(1), H(1), CNOT(0,1), CustomGate(1), H(1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 7

In [14]:
circ = QCircuit(6)
circ.add(H(1))
circ.add(S(1))
for i in range(1, 6):
    circ.add(CNOT(0, i))
for i in range(5, 0, -1):
    circ.add(CNOT(0, i))
circ.add(Sdg(1))
circ.add(H(1))

optimise_circuit(circ, p_graphs)

Circuit:
                                                                            
q_0: ────────────■────■────■────■────■────■────■────■────■────■─────────────
     ┌───┐┌───┐┌─┴─┐  │    │    │    │    │    │    │    │  ┌─┴─┐┌────┐┌───┐
q_1: ┤ H ├┤ S ├┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼──┤ X ├┤ S† ├┤ H ├
     └───┘└───┘└───┘┌─┴─┐  │    │    │    │    │    │  ┌─┴─┐└───┘└────┘└───┘
q_2: ───────────────┤ X ├──┼────┼────┼────┼────┼────┼──┤ X ├────────────────
                    └───┘┌─┴─┐  │    │    │    │  ┌─┴─┐└───┘                
q_3: ────────────────────┤ X ├──┼────┼────┼────┼──┤ X ├─────────────────────
                         └───┘┌─┴─┐  │    │  ┌─┴─┐└───┘                     
q_4: ─────────────────────────┤ X ├──┼────┼──┤ X ├──────────────────────────
                              └───┘┌─┴─┐┌─┴─┐└───┘                          
q_5: ──────────────────────────────┤ X ├┤ X ├───────────────────────────────
                                   └───┘└───┘                      

QCircuit([CustomGate(1), S(1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 8

In [15]:
circ = QCircuit(6)
circ.add(H(0))
circ.add(S(0))
for i in range(1, 6):
    circ.add(CNOT(i, 0))
for i in range(5, 0, -1):
    circ.add(CNOT(i, 0))
circ.add(Sdg(0))
circ.add(H(0))

optimise_circuit(circ, p_graphs)

Circuit:
     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌────┐┌───┐
q_0: ┤ H ├┤ S ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ S† ├┤ H ├
     └───┘└───┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└────┘└───┘
q_1: ────────────■────┼────┼────┼────┼────┼────┼────┼────┼────■─────────────
                      │    │    │    │    │    │    │    │                  
q_2: ─────────────────■────┼────┼────┼────┼────┼────┼────■──────────────────
                           │    │    │    │    │    │                       
q_3: ──────────────────────■────┼────┼────┼────┼────■───────────────────────
                                │    │    │    │                            
q_4: ───────────────────────────■────┼────┼────■────────────────────────────
                                     │    │                                 
q_5: ────────────────────────────────■────■─────────────────────────────────
                                                                   

QCircuit([CustomGate(0), S(0)], nb_qubits=1, nb_cbits=None, label="None")

# Circuit 9

In [16]:
circ = QCircuit(2)
circ.add(T(1))
circ.add(H(1))
circ.add(CNOT(0, 1))
circ.add(H(1))
circ.add(Tdg(1))

optimise_circuit(circ)

Circuit:
                               
q_0: ────────────■─────────────
     ┌───┐┌───┐┌─┴─┐┌───┐┌────┐
q_1: ┤ T ├┤ H ├┤ X ├┤ H ├┤ T† ├
     └───┘└───┘└───┘└───┘└────┘
Optimised circuit
                    
q_0: ───────■───────
     ┌───┐┌─┴─┐┌───┐
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘


/home/michalon/epita/ing3/pfee/pyZX/pfee_colibritd/.venv/lib/python3.10/site-packages/mpqp/core/instruction/gates/gate_definition.py:107: UserWarning: Cannot ensure that a operator defined with variables is unitary.
  warn("Cannot ensure that a operator defined with variables is unitary.")


QCircuit([H(1), CNOT(0,1), H(1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 10

In [17]:
circ = QCircuit(2)
circ.add(T(1))
circ.add(CNOT(0, 1))
circ.add(S(1))
circ.add(CNOT(0, 1))
circ.add(Tdg(1))

optimise_circuit(circ)

Circuit:
                               
q_0: ───────■─────────■────────
     ┌───┐┌─┴─┐┌───┐┌─┴─┐┌────┐
q_1: ┤ T ├┤ X ├┤ S ├┤ X ├┤ T† ├
     └───┘└───┘└───┘└───┘└────┘
Optimised circuit
                    
q_0: ──■─────────■──
     ┌─┴─┐┌───┐┌─┴─┐
q_1: ┤ X ├┤ S ├┤ X ├
     └───┘└───┘└───┘


QCircuit([CNOT(0,1), S(1), CNOT(0,1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 11

In [18]:
circ = QCircuit(2)
circ.add(T(0))
circ.add(CNOT(0, 1))
circ.add(Tdg(0))

optimise_circuit(circ)

Circuit:
     ┌───┐     ┌────┐
q_0: ┤ T ├──■──┤ T† ├
     └───┘┌─┴─┐└────┘
q_1: ─────┤ X ├──────
          └───┘      
Optimised circuit
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘


QCircuit([CNOT(0,1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 12

In [19]:
circ = QCircuit(3)

circ.add(T(1))

# Mixture of first, second and third patterns
circ.add(H(1))
circ.add(CNOT(0, 1))
circ.add(H(1))

circ.add(CNOT(0, 1))
circ.add(S(1))
circ.add(CNOT(0, 1))

circ.add(H(1))
circ.add(CNOT(0, 1))
circ.add(H(1))

#circ.add(CNOT(1, 2))
#circ.add(CNOT(1, 2))

circ.add(Tdg(1))

optimise_circuit(circ)

Circuit:
                                                             
q_0: ────────────■─────────■─────────■─────────■─────────────
     ┌───┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌────┐
q_1: ┤ T ├┤ H ├┤ X ├┤ H ├┤ X ├┤ S ├┤ X ├┤ H ├┤ X ├┤ H ├┤ T† ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└────┘
q_2: ────────────────────────────────────────────────────────
                                                             
Optimised circuit
                                                  
q_0: ───────■─────────■─────────■─────────■───────
     ┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐
q_1: ┤ H ├┤ X ├┤ H ├┤ X ├┤ S ├┤ X ├┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘


QCircuit([H(1), CNOT(0,1), H(1), CNOT(0,1), S(1), CNOT(0,1), H(1), CNOT(0,1), H(1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 13

In [20]:
circ = QCircuit(3)
circ.add(CNOT(0, 2))
circ.add(CNOT(1, 2))
circ.add(CNOT(0, 2))

optimise_circuit(circ)

Circuit:
                    
q_0: ──■─────────■──
       │         │  
q_1: ──┼────■────┼──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_2: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘
Optimised circuit
          
q_0: ─────
          
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘


QCircuit([CNOT(1,2)], nb_qubits=3, nb_cbits=None, label="None")

# Circuit 14

In [21]:
circ = QCircuit(3)
circ.add(CNOT(0, 2))
circ.add(CNOT(0, 1))
circ.add(CNOT(0, 2))

optimise_circuit(circ)

Circuit:
                    
q_0: ──■────■────■──
       │  ┌─┴─┐  │  
q_1: ──┼──┤ X ├──┼──
     ┌─┴─┐└───┘┌─┴─┐
q_2: ┤ X ├─────┤ X ├
     └───┘     └───┘
Optimised circuit
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘


QCircuit([CNOT(0,1)], nb_qubits=2, nb_cbits=None, label="None")

# Circuit 15

In [22]:
circ = QCircuit(3)

circ.add(CNOT(0, 1))
circ.add(H(1))
circ.add(CNOT(1, 2))
circ.add(H(1))
circ.add(CNOT(0, 1))

optimise_circuit(circ)

Circuit:
                              
q_0: ──■───────────────────■──
     ┌─┴─┐┌───┐     ┌───┐┌─┴─┐
q_1: ┤ X ├┤ H ├──■──┤ H ├┤ X ├
     └───┘└───┘┌─┴─┐└───┘└───┘
q_2: ──────────┤ X ├──────────
               └───┘          
Optimised circuit
                    
q_0: ───────────────
     ┌───┐     ┌───┐
q_1: ┤ H ├──■──┤ H ├
     └───┘┌─┴─┐└───┘
q_2: ─────┤ X ├─────
          └───┘     


QCircuit([H(1), CNOT(1,2), H(1)], nb_qubits=3, nb_cbits=None, label="None")

# Circuit 16

In [23]:
circ = QCircuit(4)
circ.add(CNOT(0, 2))

circ.add(CNOT(1, 2))

circ.add(CNOT(0, 1))

circ.add(H(2))
circ.add(CNOT(2, 3))
circ.add(H(2))

circ.add(CNOT(0, 2))

optimise_circuit(circ)

Circuit:
                                   
q_0: ──■─────────■──────────────■──
       │       ┌─┴─┐            │  
q_1: ──┼────■──┤ X ├────────────┼──
     ┌─┴─┐┌─┴─┐├───┤     ┌───┐┌─┴─┐
q_2: ┤ X ├┤ X ├┤ H ├──■──┤ H ├┤ X ├
     └───┘└───┘└───┘┌─┴─┐└───┘└───┘
q_3: ───────────────┤ X ├──────────
                    └───┘          
Optimised circuit
                         
q_0: ───────■────────────
          ┌─┴─┐          
q_1: ──■──┤ X ├──────────
     ┌─┴─┐├───┤     ┌───┐
q_2: ┤ X ├┤ H ├──■──┤ H ├
     └───┘└───┘┌─┴─┐└───┘
q_3: ──────────┤ X ├─────
               └───┘     


QCircuit([CNOT(1,2), CNOT(0,1), H(2), CNOT(2,3), H(2)], nb_qubits=4, nb_cbits=None, label="None")